In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')

In [2]:
def get_StockInfo(quote):
    financeSource = 'https://www.google.com.au/search?q=ASX:{'+quote+'}&num=20&tbm=nws&source=lnt&tbs=sbd:1&sa=X&ved=0ahUKEwico5WXm9HXAhUMGJAKHd-CBu8QpwUIHw&biw=1167&bih=539&dpr=1.65'
    page = requests.get(financeSource)
    bs = BeautifulSoup(page.content,"html.parser")
    tables = bs.find_all('table')
    
    titles = []
    dtimes = []
    descs = []
    urls = []
    
    for i in range(3,len(tables)-1):
        tbl = tables[i].find('a') # url link in each table
        dtime = tables[i].find('span') #publish datetime of url 
        title = tbl.text #article title
        desc = tables[i].find_all('div')[1].text #description of link
        link = "https://www.google.com.au/" + tbl.get('href') #link from the google search
        titles.append(title)
        dtimes.append(dtime.text)
        descs.append(desc)
        urls.append(link)
        
    df = pd.DataFrame()
    df['Title'] = titles
    df['PubDate'] = dtimes
    df['URL'] = urls
    #     df['Description'] = descs
    
    df[['Source', 'PubDate']] = df['PubDate'].str.split(' - ', expand=True)
    df = df[['Title', 'PubDate', 'Source', 'URL']]
    
    return df

In [3]:
def extract_text(link):
    text = ""
    page = requests.get(link)
    soup = BeautifulSoup(page.content,'html.parser')
    title = soup.title.text.replace('\t','')
    title = title.replace('\n','')
    content = soup.find_all('p')
    for p in content:
        para = p.text.replace('\t','')
        para = para.replace('\n','')
        text += para + "\n"
    return (title + "\n" + text)

In [54]:
df = get_StockInfo('airtel')

In [55]:
text=[]

In [56]:
for i in df['URL']:
    text.append(extract_text(i))

In [57]:
df['Article']=text

In [62]:
import spacy
nlp=spacy.load('en')

In [59]:
for i in range(0,len(text)):
    count=[]
    wikitext=nlp(text[i])
    for word in wikitext.ents:
        if word.label_=='ORG':
            count.append(word.text.lower())
    if sum((itm.count("airtel") for itm in count))<3:
        df=df.drop(i)

In [60]:
df = df.reset_index(drop=True)

In [61]:
df

,Title,PubDate,Source,URL,Article
0,"Sensex closes 329 points up, Nifty at 11946, N...",30 May 2019,Business Today,https://www.google.com.au//url?q=https://www.b...,"Sensex closes 329 points up, Nifty at 11,946, ..."
1,"Sensex rises over 150 points, Nifty crosses 11...",21 Mar 2019,Business Today,https://www.google.com.au//url?q=https://www.b...,"Sensex rises over 150 points, Nifty crosses 11..."
2,"Sensex closes 481 points higher, Nifty above 1...",12 Mar 2019,Business Today,https://www.google.com.au//url?q=https://www.b...,"Sensex closes 481 points higher, Nifty above 1..."
3,Why Bharti Airtel Limited's (NSE:BHARTIARTL) C...,30 Oct 2018,Simply Wall St,https://www.google.com.au//url?q=https://simpl...,Why Bharti Airtel Limited’s (NSE:BHARTIARTL) C...
4,What You Must Know About Bharti Airtel Limited...,20 Aug 2018,Simply Wall St,https://www.google.com.au//url?q=https://simpl...,What You Must Know About Bharti Airtel Limited...
5,"Sensex hits all-time high of 37533, Nifty cros...",30 Jul 2018,Business Today,https://www.google.com.au//url?q=https://www.b...,"Sensex hits all-time high of 37,533, Nifty cro..."
6,Is It Too Late To Buy Bharti Airtel Limited (N...,8 May 2018,Simply Wall St,https://www.google.com.au//url?q=https://simpl...,Is It Too Late To Buy Bharti Airtel Limited (N...
7,Is Bharti Airtel Limited (NSE:BHARTIARTL) Wort...,19 Apr 2018,Simply Wall St,https://www.google.com.au//url?q=https://simpl...,Is Bharti Airtel Limited (NSE:BHARTIARTL) Wort...
8,Is Bharti Airtel Limited's (NSE:BHARTIARTL) Gr...,10 Apr 2018,Simply Wall St,https://www.google.com.au//url?q=https://simpl...,Is Bharti Airtel Limited’s (NSE:BHARTIARTL) Gr...


In [64]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [65]:
sia =SentimentIntensityAnalyzer()

In [72]:
df_score=pd.DataFrame(columns=['sentiment_score'])

In [73]:
for i in df['Article']:
    df_score=df_score.append({'sentiment_score':sia.polarity_scores(i)['compound']},ignore_index=True)

In [74]:
df_score = pd.concat([df[['Title','PubDate','Source']], df_score], axis=1)

In [75]:
df_score

,Title,PubDate,Source,sentiment_score
0,"Sensex closes 329 points up, Nifty at 11946, N...",30 May 2019,Business Today,0.9972
1,"Sensex rises over 150 points, Nifty crosses 11...",21 Mar 2019,Business Today,0.9923
2,"Sensex closes 481 points higher, Nifty above 1...",12 Mar 2019,Business Today,0.9980
3,Why Bharti Airtel Limited's (NSE:BHARTIARTL) C...,30 Oct 2018,Simply Wall St,0.9533
4,What You Must Know About Bharti Airtel Limited...,20 Aug 2018,Simply Wall St,0.9878
5,"Sensex hits all-time high of 37533, Nifty cros...",30 Jul 2018,Business Today,0.9977
6,Is It Too Late To Buy Bharti Airtel Limited (N...,8 May 2018,Simply Wall St,0.9993
7,Is Bharti Airtel Limited (NSE:BHARTIARTL) Wort...,19 Apr 2018,Simply Wall St,0.9990
8,Is Bharti Airtel Limited's (NSE:BHARTIARTL) Gr...,10 Apr 2018,Simply Wall St,0.9852


In [76]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [77]:
stopwords_list = stopwords.words('english')
stopwords_set = set(stopwords_list)
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [94]:
def get_final_tokens(article):
    article = article.lower()
    tokenizer = RegexpTokenizer(r"\w+(?:[-.]\w+)?")
    tagged_sents = []
    sentences = sent_detector.tokenize(article.strip())
    for sent in sentences:
        uni_sent = tokenizer.tokenize(sent)
        tagged_sent = nltk.tag.pos_tag(uni_sent)
        stopped_tagged_sent = [x for x in tagged_sent if x[0] not in stopwords_set]  
        tagged_sents.append(stopped_tagged_sent)
    lemmatizer = WordNetLemmatizer()
    final_tokens=[]
    for tagged_set in tagged_sents:
        final_tokens = final_tokens + [lemmatizer.lemmatize(w[0], get_wordnet_pos(w[1]))for w in tagged_set]
    return final_tokens

In [128]:
final_article=[]

In [129]:
for i in df['Article']:
    final_article.append(' '.join(get_final_tokens(i)))

In [133]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.datasets import load_files
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [136]:
x=final_article
y=[1,1,1,1,1,1,1,1,1]

In [137]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [141]:
vectorizer=TfidfVectorizer(min_df=5,max_df=0.8,sublinear_tf=True,use_idf=True,stop_words='english')

In [143]:
train_corpous_tf_idf=vectorizer.fit_transform(x_train)
test_corpous_tf_idf=vectorizer.transform(x_test)

/usr/lib/python3/dist-packages/sklearn/feature_extraction/text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [144]:
model=MultinomialNB()

In [145]:
model.fit(train_corpous_tf_idf,y_train)

/usr/lib/python3/dist-packages/sklearn/naive_bayes.py:420: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [146]:
result=model.predict(test_corpous_tf_idf)

In [147]:
result

array([1, 1])